In [ ]:
import pandas as pd
import requests
import logging
import json
import time

logging.basicConfig(level=logging.INFO)

def getData(page, tag):

    base_url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "order": "asc",
        "sort": "creation",
        "tagged": tag,
        "site": "stackoverflow",
        "pagesize": 100,
        "page": page,
        "filter": "withbody",
        "key": "rl_7G8XRSxnP5ZMRRFLme9xurK36"
    }

    try:
        response = requests.get(base_url, params=params,timeout=(10, 10))
        response.raise_for_status()
        data = response.json()

        if "items" not in data or not data["items"]:
            logging.warning("No questions found.")
            return None

        data_list = []
        for question in data["items"]:
            accepted_answer=''
            accepted_answer_score=0
            answer_id = question.get("accepted_answer_id",None)
            time.sleep(0.1)
            if answer_id:
                answers_url = f"https://api.stackexchange.com/2.3/answers/{answer_id}"
                answers_params = {
                    "site": "stackoverflow",
                    "filter": "withbody",
                    "key": "rl_7G8XRSxnP5ZMRRFLme9xurK36" 
                }
                answers_response = requests.get(answers_url, params=answers_params, timeout=(10, 10))
                answers_response.raise_for_status()
                answer = answers_response.json().get('items', [])[0]

                accepted_answer=answer.get('body', '')
                accepted_answer_score=answer.get('score', 0)
    
            data_list.append([
                question.get('title', ''),
                question.get('body', ''),
                ", ".join(question.get('tags', [])),
                accepted_answer,
                accepted_answer_score,
                question.get('score', 0),
                question.get('view_count', 0),
                pd.to_datetime(question.get('creation_date'), unit='s'),
                question.get('link','')
            ])


        df = pd.DataFrame(data_list, columns=["Title", "Description", "Tags", "Accepted Answer", "Answer Score", "Question Score", "Question Views", "Creation Time", "Link"])
        
        file_name = "questions_data_nlp.csv"

        with open(file_name, 'a', encoding='utf-8', newline='') as f:
              # Write header only if file is empty
              df.to_csv(f, index=False)
        # logging.info(f"Data appended {count}")

    except requests.exceptions.RequestException as req_e:
        logging.error(f"Request failed: {req_e}")
    except json.JSONDecodeError as json_e:
         logging.error(f"Failed to decode JSON response: {json_e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
    return None


In [ ]:
for page in range(1,100):
    result = getData(page,'nlp')
    print(page)